In [1]:
import syft as sy 
import torch 
from torchvision import datasets 
from torchvision import transforms

In [30]:
duet  = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...



In [3]:
def split_data(dataset, worker_list=None, n_workers=2):

    if worker_list is None:
        worker_list = list(range(0, n_workers))
            
    #counter to create the index of different data samples
    idx = 0 
    
    #dictionary to accomodate the split data
    dic_single_datasets = {}
    for worker in worker_list: 
        """
        Each value is a list of three elements, to accomodate, in order: 
        - data examples (as tensors)
        - label
        - index 
        """
        dic_single_datasets[worker] = [] 

    """
    Loop through the dataset to split the data and labels vertically across workers. 
    Splitting method from @abbas5253: https://github.com/abbas5253/SplitNN-for-Vertically-Partitioned-Data/blob/master/distribute_data.py
    """
    
    label_list = []
    index_list = []
    for tensor, label in dataset:
        height = tensor.shape[-1]//len(worker_list)
        i = 0
        for worker in worker_list[:-1]: 
            dic_single_datasets[worker].append(tensor[:, :, height * i : height * (i + 1)])
            i += 1
            
        #add the value of the last worker / split
        dic_single_datasets[worker_list[-1]].append(tensor[:, :, height * (i) : ])
        label_list.append(torch.Tensor([label]))
        index_list.append(torch.Tensor([idx]))
        
        idx += 1
        
    return dic_single_datasets, label_list, index_list

In [6]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset = datasets.MNIST('mnist', download=True, train=True, transform=transform)
valset = datasets.MNIST('mnist', download=True, train=False, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable


100.0%



Failed to download (trying next):
HTTP Error 503: Service Unavailable



102.8%

Failed to download (trying next):
HTTP Error 503: Service Unavailable



...


Failed to download (trying next):
HTTP Error 503: Service Unavailable



112.7%

Extracting mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist\MNIST\raw

Processing...



C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [12]:
img , _ , _ = split_data(trainset)

{0: [], 1: []}

In [17]:
img = torch.cat(img[0][:25_000])

In [18]:
img_rshp= img.view(img.shape[0],-1)

In [23]:
val_img,_,_ = split_data(valset)
val_img = torch.cat(val_img[0][:5_000])
val_img_rshp = val_img.view(val_img.shape[0],-1)


{0: [], 1: []}


In [32]:
img_rshp.send(duet,searchable=True,tags=["DO1"],description="This the MNIST dataset comming from the DO1 and contains 25_000 img")
val_img.send(duet,searchable=True,tags=["DO1_test"],description="This the MNIST dataset (test) comming from the DO1 and contains 5_000 img")

`searchable` is deprecated please use `pointable` in future

In [33]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 448e8111d76d4a9d8b85d3ce4f396f53>,[DO1],This the MNIST dataset comming from the DO1 an...,<class 'torch.Tensor'>
1,<UID: 9e60210e833748589481ffb03600f220>,[DO1_test],This the MNIST dataset (test) comming from the...,<class 'torch.Tensor'>


[2021-08-14T02:39:39.843918+0100][CRITICAL][logger]][17964] You do not have permission to .get() Object with ID: <UID: 448e8111d76d4a9d8b85d3ce4f396f53>Please submit a request.
[2021-08-14T02:39:39.844918+0100][CRITICAL][logger]][17964] You do not have permission to .get() Object with ID: <UID: 448e8111d76d4a9d8b85d3ce4f396f53>Please submit a request.


In [39]:
duet.requests.add_handler(action="accept")

C:\Users\21355\Anaconda3\envs\pysyft\lib\site-packages\syft\lib\torch\uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)
